### Import libraries

In [1]:
import supervision as sv
import ultralytics
import numpy as np
import os
from IPython import display
from PIL import Image
from datetime import datetime
display.clear_output()
ultralytics.checks()

Ultralytics YOLOv8.0.208 🚀 Python-3.11.5 torch-2.1.0+cu121 CPU (AMD Ryzen 7 5800H with Radeon Graphics)
Setup complete ✅ (16 CPUs, 15.0 GB RAM, 366.0/468.1 GB disk)


### Set paths

In [2]:
model = "train10" # Model that we're going to use
cwd = os.getcwd()
# foto que vamos a procesar
video_path = os.path.join(cwd, "assets", "dump-truck-loading-material.mp4")

# ubicacion del modelo base de yolo
model_path = os.path.join(cwd, "runs", "detect", model, "weights", "best.pt")

# ubicacion de salida
output_path = os.path.join(cwd, "output")

# ubicacion de video de salida ya con anotaciones
output_video_path = os.path.join(output_path, "dump-truck-loading-material-annotated.mp4")

### Test the model (Video)

In [6]:

# cargar modelo
model = ultralytics.YOLO(model_path)

# anotador que nos permite aplicar las predicciones a la imagen para visualizarlas
annotator = sv.BoxAnnotator()

# byte tracker nos permite mantener track de los objetos que se van detectando en el video
byte_tracker = sv.ByteTrack()

# esta funcion se manda llamar por cada frame del video
all_results = []
n = 0
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # inferencia y obtener predicciones
    results = model(frame)

    # como solo tenemos un frame, solo tomamos el primer resultado de la lista
    first_frame_results = results[0]

    all_results.append(first_frame_results) 
    
    # convertir las predicciones a un objeto de supervision que servira para visualizarlas
    detections = sv.Detections.from_ultralytics(first_frame_results)
    detections = byte_tracker.update_with_detections(detections)

    
    # agregar etiquetas para facilitar saber que se esta detectando
    labels = [
        # imprimimos la clase en texto y la confianza que el modelo tiene en su prediccion
        f"{model.model.names[class_id]} {confidence:0.2f}"
        for _, _, confidence, class_id, _
        in detections
    ]
    n++;
    return annotator.annotate(scene=frame.copy(), detections=detections, labels=labels)

sv.process_video(
    source_path=video_path,
    target_path=output_video_path,
    callback=callback,
)

UnboundLocalError: cannot access local variable 'n' where it is not associated with a value

In [5]:
for r in all_results:
    box = r.boxes[0]
    if (len(r.boxes) >= 1):
        if (box.cls[0] == 0): # Empty truck
            print(f"{r.names[0]} with {round(box.conf[0].item(), 4)} percent of confidence")
        
        elif (box.cls[0] == 1): # Filled truck
            print(f"{r.names[0]} with {round(box.conf[0].item(), 4)} percent of confidence")
    
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    
    current_date = datetime.now().strftime('%Y%m%d-%H:%M')
    im.save(f"output/{current_date}.jpg")  # save image


empty-truck with 0.3024 percent of confidence
empty-truck with 0.3539 percent of confidence
empty-truck with 0.3032 percent of confidence
empty-truck with 0.2968 percent of confidence
empty-truck with 0.3287 percent of confidence
empty-truck with 0.359 percent of confidence
empty-truck with 0.3249 percent of confidence
empty-truck with 0.3016 percent of confidence
empty-truck with 0.3265 percent of confidence
empty-truck with 0.3091 percent of confidence
empty-truck with 0.2987 percent of confidence
empty-truck with 0.2879 percent of confidence
empty-truck with 0.2991 percent of confidence
empty-truck with 0.3239 percent of confidence
empty-truck with 0.3588 percent of confidence
empty-truck with 0.3227 percent of confidence
empty-truck with 0.3222 percent of confidence
empty-truck with 0.4334 percent of confidence
empty-truck with 0.4434 percent of confidence
empty-truck with 0.4791 percent of confidence
empty-truck with 0.3422 percent of confidence
empty-truck with 0.4944 percent of 

IndexError: index 0 is out of bounds for dimension 0 with size 0

### Test the model (Image)

In [53]:

# cargar modelo
model = ultralytics.YOLO(model_path)

source = './assets/img1.png'

results = model(source)

for r in results:
    box = r.boxes[0]
    if (len(r.boxes) >= 1):
        if (box.cls[0] == 0): # Empty truck
            print(f"{r.names[0]} with {round(box.conf[0].item(), 4)} percent of confidence")
        
        elif (box.cls[0] == 1): # Filled truck
            print(f"{r.names[0]} with {round(box.conf[0].item(), 4)} percent of confidence")
    
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    
    current_date = datetime.now().strftime('%Y%m%d-%H:%M')
    im.save(f"output/{current_date}.jpg")  # save image


image 1/1 /home/esquivelg/Documents/GitHub/reto-banco-materiales/assets/img1.png: 384x640 2 empty-trucks, 78.0ms
Speed: 2.0ms preprocess, 78.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


empty-truck with 0.3285 percent of confidence
